In [1]:
import sys 
import numpy as np
import pandas as pd

sys.path.append(r'C:\Users\DerDo\Desktop\fantasy_basketball_project')
from scrape_functions import *
from database_conn import SQLiteClient

scraper = Scrape_Functions()
db_path = r"D:\nba.db"

In [2]:
np.array(scraper.teams)

array(['ATL', 'BOS', 'NJN', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW',
       'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOH',
       'NYK', 'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', 'TOR',
       'UTA', 'WAS', 'CHA'], dtype='<U3')

Push Historical Team Data

In [3]:
all_team_data = scraper.all_team_historical_data()

new_cols = [
    col.lower().replace('/', '').replace('%', '') for col in all_team_data.columns
]
new_cols[2] = 'start'
new_cols[3] = 'end'

all_team_data.columns = new_cols

all_team_data = all_team_data.iloc[
    all_team_data['franchise'].drop_duplicates(keep='first').index
].reset_index(drop=True)

all_team_data['start'] = (
    all_team_data
    ['start']
    .apply(
        lambda x: 
        x[0:4] if len(x) != 0 else ""
    )
)

all_team_data['end'] = (
    all_team_data
    ['end']
    .apply(
        lambda x: 
        x[0:2] + x[-2:] if len(x) != 0 and x[0:4] != '1999' else '2000'
    )
)

# Create or use the DB
with SQLiteClient(db_path) as db:
    # 1) Create a table
    db.create_table(
        "historical_team_data",
        {
            "franchise": "TEXT PRIMARY KEY",
            "lg": "TEXT NULL",
            "start": "TEXT NULL",
            "end": "TEXT NULL",
            "yrs": "TEXT NULL",
            "g": "TEXT NULL",
            "w": "TEXT NULL",
            "l": "TEXT NULL",
            "wl": "TEXT NULL",
            'plyfs': "TEXT NULL",
            'div': 'TEXT NULL',
            'conf': 'TEXT NULL',
            'champ': 'TEXT NULL'
        }
    )

    # Upsert dataframe
    db.upsert_dataframe(df=all_team_data, table='historical_team_data', conflict_cols='franchise')

    # Create the new table with the desired schema
    db.execute("""
        CREATE TABLE IF NOT EXISTS _historical_team_data_new (
            franchise TEXT PRIMARY KEY,
            lg        TEXT,
            "start"   INTEGER,
            "end"     INTEGER,
            yrs       INTEGER,
            g         INTEGER,
            w         INTEGER,
            l         INTEGER,
            wl        REAL,
            plyfs     INTEGER,
            div       INTEGER,
            conf      INTEGER,
            champ     INTEGER
        );
    """)

    # Copy data from the old table into the new one
    db.execute("""
        INSERT INTO _historical_team_data_new
            (franchise, lg, "start", "end", yrs, g, w, l, wl, plyfs, div, conf, champ)
        SELECT
            franchise, lg, "start", "end", yrs, g, w, l, wl, plyfs, div, conf, champ
        FROM historical_team_data;
    """)

    # Swap tables
    db.execute("DROP TABLE historical_team_data;")
    db.execute('ALTER TABLE _historical_team_data_new RENAME TO historical_team_data;')

    # Query the data
    data = db.query("SELECT * FROM historical_team_data")



In [44]:
pd.DataFrame(data).sort_values('end', ascending=False)

,franchise,lg,start,end,yrs,g,w,l,wl,plyfs,div,conf,champ
0,Atlanta Hawks,NBA,1949,2026,77,6019,2967,3052,0.493,49,12,0,1
5,Brooklyn Nets,NBA/ABA,1967,2026,59,4694,2054,2640,0.438,31,5,2,2
4,Boston Celtics,NBA/BAA,1946,2026,80,6196,3695,2501,0.596,62,35,11,18
12,Cleveland Cavaliers,NBA,1970,2026,56,4435,2096,2339,0.473,25,8,5,1
13,Dallas Mavericks,NBA,1980,2026,46,3625,1836,1789,0.506,25,5,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,Minneapolis Lakers,NBA/BAA,1948,1960,12,839,457,382,0.545,11,4,0,5
17,Fort Wayne Pistons,NBA/BAA,1948,1957,9,619,313,306,0.506,8,2,0,0
49,Rochester Royals,NBA/BAA,1948,1957,9,620,357,263,0.576,7,2,0,1
2,Milwaukee Hawks,NBA,1951,1955,4,281,91,190,0.324,0,0,0,0


**BEFORE RUNNING THESE WAIT A COUPLE OF SECONDS IN BETWEEN RUNS**

---

In [5]:
atl = scraper.team_historical_data('ATL')
bos = scraper.team_historical_data('BOS')
brk = scraper.team_historical_data('NJN')
chi = scraper.team_historical_data('CHI')
cle = scraper.team_historical_data('CLE')

In [6]:
dal = scraper.team_historical_data('DAL')
den = scraper.team_historical_data('DEN')
det = scraper.team_historical_data('DET')
gsw = scraper.team_historical_data('GSW')
hou = scraper.team_historical_data('HOU')

In [7]:
ind = scraper.team_historical_data('IND')
lac = scraper.team_historical_data('LAC')
lal = scraper.team_historical_data('LAL')
mem = scraper.team_historical_data('MEM')
mia = scraper.team_historical_data('MIA')

In [8]:
mil = scraper.team_historical_data('MIL')
min = scraper.team_historical_data('MIN')
nop = scraper.team_historical_data('NOH')
nyk = scraper.team_historical_data('NYK')
okc = scraper.team_historical_data('OKC')

In [9]:
orl = scraper.team_historical_data('ORL')
phi = scraper.team_historical_data('PHI')
pho = scraper.team_historical_data('PHO')
por = scraper.team_historical_data('POR')
sac = scraper.team_historical_data('SAC')

In [10]:
sas = scraper.team_historical_data('SAS')
tor = scraper.team_historical_data('TOR')
uta = scraper.team_historical_data('UTA')
was = scraper.team_historical_data('WAS')
cho = scraper.team_historical_data('CHA')

---

historical_team_data is data per team, team_historical_data is data per active team

In [58]:
team_data = [
    atl, bos, brk, chi, cle, dal, den, det, gsw, hou, ind, lac, lal, mem, mia,
    mil, min, nop, nyk, okc, orl, phi, pho, por, sac, sas, tor, uta, was, cho
]

team_data = (
    pd
    .concat(team_data, axis=0)
    .replace('', None)
    .drop(columns='')
)

new_cols = [
    col.lower().replace('/', '').replace('%', '').replace(' ', '') for col in team_data.columns
]

team_data.columns = new_cols

team_data['primarykey'] = team_data['season'] + team_data['team']

with SQLiteClient(db_path) as db:
    # 1) Create a table
    db.create_table(
        "active_historical_team_data",
        {
            "season": "TEXT NULL",
            "lg": "TEXT NULL",
            "team": "TEXT NULL",
            "w": "TEXT NULL",
            "l": "TEXT NULL",
            "wl": "TEXT NULL",
            "finish": "TEXT NULL",
            "srs": "TEXT NULL",
            "pace": "TEXT NULL",
            "relpace": "TEXT NULL",
            "ortg": "TEXT NULL",
            "relortg": "TEXT NULL",
            "drtg": "TEXT NULL",
            "reldrtg": "TEXT NULL",
            "playoffs": "TEXT NULL",
            "coaches": "TEXT NULL",
            "topws": "TEXT NULL",
            "primarykey": "TEXT PRIMARY KEY"
        }
    )

    db.upsert_dataframe(
        df=team_data, 
        table='active_historical_team_data', 
        conflict_cols='primarykey'
    )

    db.execute("""
        CREATE TABLE IF NOT EXISTS _active_historical_team_data_new (
            season           TEXT,
            lg               TEXT,
            team             TEXT,
            w                INTEGER,
            l                INTEGER,
            wl               REAL,
            finish           TEXT,
            srs              REAL,
            pace             REAL,
            "relpace"       REAL,
            ortg             REAL,
            "relortg"       REAL,
            drtg             REAL, 
            "reldrtg"       REAL,
            playoffs         TEXT,
            coaches          TEXT,
            "topws"         TEXT,
            "primarykey"    TEXT PRIMARY KEY
        );
    """)

    db.execute("""
        INSERT INTO _active_historical_team_data_new
            (season, lg, team, w, l, wl, finish, srs, pace, "relpace",
             ortg, "relortg", drtg, "reldrtg", playoffs, coaches, "topws", "primarykey")
        SELECT
            season, lg, team, w, l, wl, finish, srs, pace, "relpace",
            ortg, "relortg", drtg, "reldrtg", playoffs, coaches, "topws", "primarykey"
        FROM active_historical_team_data;
    """)

    db.execute("DROP TABLE active_historical_team_data;")
    db.execute('ALTER TABLE _active_historical_team_data_new RENAME TO active_historical_team_data;')

    data = db.query("SELECT * FROM active_historical_team_data")

In [59]:
pd.DataFrame(data)

,season,lg,team,w,l,wl,finish,srs,pace,relpace,ortg,relortg,drtg,reldrtg,playoffs,coaches,topws,primarykey
0,2025-26,NBA,Atlanta Hawks,0,0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,2025-26Atlanta Hawks
1,2024-25,NBA,Atlanta Hawks,40,42,0.488,2nd of 5,-1.41,102.6,3.8,114.6,0.1,115.7,1.2,None,Q. Snyder(40-42),O. Okongwu(7.2),2024-25Atlanta Hawks
2,2023-24,NBA,Atlanta Hawks,36,46,0.439,3rd of 5,-2.38,100.1,1.6,117.2,1.9,119.4,4.1,None,Q. Snyder(36-46),C. Capela(6.3),2023-24Atlanta Hawks
3,2022-23,NBA,Atlanta Hawks*,41,41,0.500,2nd of 5,0.32,100.7,1.6,116.6,1.8,116.3,1.5,Lost E. Conf. 1st Rnd.,"N. McMillan(29-30),J. Prunty(2-0),Q. Snyder(10...",C. Capela(7.2),2022-23Atlanta Hawks*
4,2021-22,NBA,Atlanta Hawks*,43,39,0.524,2nd of 5,1.55,97.7,-0.5,116.5,4.5,114.9,2.9,Lost E. Conf. 1st Rnd.,N. McMillan(43-39),T. Young(10.0),2021-22Atlanta Hawks*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1718,1992-93,NBA,Charlotte Hornets*,44,38,0.537,3rd of 7,-0.02,100.2,3.4,109.5,1.5,109.7,1.7,Lost E. Conf. Semis,A. Bristow(44-38),L. Johnson(9.6),1992-93Charlotte Hornets*
1719,1991-92,NBA,Charlotte Hornets,31,51,0.378,7th of 7,-3.57,101.7,5.1,107.1,-1.1,110.9,2.7,None,A. Bristow(31-51),L. Johnson(8.9),1991-92Charlotte Hornets
1720,1990-91,NBA,Charlotte Hornets,26,56,0.317,7th of 7,-4.95,97.3,-0.5,105.0,-2.9,110.4,2.5,None,G. Littles(26-56),M. Bogues(4.8),1990-91Charlotte Hornets
1721,1989-90,NBA,Charlotte Hornets,19,63,0.232,7th of 7,-7.00,98.1,-0.2,102.0,-6.1,109.9,1.8,None,"D. Harter(8-32),G. Littles(11-31)",M. Bogues(6.7),1989-90Charlotte Hornets
